In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [343]:
import datetime as dt

import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
from pandas.core import datetools

from sklearn.preprocessing import LabelEncoder

import numpy as np

import matplotlib.pyplot as plt

from sklearn import tree
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

In [357]:
# df_raw = pd.read_excel('c:/users/elkzze1/OneDrive - McKesson Corporation/training/ds201_capstone/data/HealthMart_Performance_Data_clean.xlsx', sheet_name='Health_Mart_Store_Progam_List_c')
# df_raw = pd.read_excel('c:/users/elkzze1/OneDrive - McKesson Corporation/training/ds201_capstone/data/HM_Master_Data_Analytics_Clean.xlsx', sheet_name='Health_Mart_Store_Progam_List_c')
# data_path = 'c:/users/elkzze1/work/ds201_capstone_data/HM_Master_Data_Analytics_Clean.xlsx' 
df_raw = pd.read_excel('c:/users/elkzze1/OneDrive - McKesson Corporation/training/ds201_capstone/data/HM_Master_Data_Analytics_Clean.xlsx', sheet_name='Health_Mart_Store_Progam_List_c', index_col=0)
df_raw.head()

,Nabp_Ncpdp,Npi_Num,Dea_Num,Target,Store_Name,Address-Delivery,City-Delivery,ST,Zip,Owner_Mgr_Full_Name,...,Profitability,Annual_Dispense_Fee_Total,Macro_PID,Script_Count,Capture_Rate,Specialty_Spend,Specialty_Product_Purchases,Generic,Brand,Generic_Ratio
Account,,,,,,,,,,,,,,,,,,,,,
10491,3800047,1205901485,AM1635474,10491.0,POSTAL PHARMACY,47809 HIGHWAY 58,OAKRIDGE,OR,97463,Laurie Patty,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17475,3802421,1912059569,AH1621336,17475.0,HOWARDS DRUGS,101 NORTH F STREET,LAKEVIEW,OR,97630,Jeff Howard,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19901,3804425,1114456563,AK1605887,19901.0,HARVARD AVENUE DRUGS,1175 HARVARD AVE,ROSEBURG,OR,97471,Philip Kaser,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
309745,3804324,1437294808,FS5689964,309745.0,REEDSPORT PHARMACY,1409 HWY 101,REEDSPORT,OR,97467,JAMAICA STANDIFORD,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
652229,3844366,1275808743,FJ3131290,652229.0,PHILOMATH FAMILY PHARMACY,1640 MAIN STREET,PHILOMATH,OR,97370,SETH PHAN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [315]:
df_raw.columns

Index(['Nabp_Ncpdp', 'Npi_Num', 'Dea_Num', 'Target', 'Store_Name',
       'Address-Delivery', 'City-Delivery', 'ST', 'Zip', 'Owner_Mgr_Full_Name',
       'Owner_Email', 'Phone_Number', 'Hm_Start_Dt', 'Channel_Type_Cd',
       'Cust_Type_Desc', 'Bus_Type_Desc', 'Region', 'DC_Name', 'PSAO_Expanded',
       'Buying_Group', 'Chain_Name', 'Sub_Group_Name', 'Region_Name',
       'Region_Director', 'Pharm_D', 'Vpgm_Name', 'Vps_Name', 'Dsm_Name',
       'RSM_Name', 'Salesperson', 'Salesperson_Group', 'AH_Program',
       'Auto_Ship_Ind', 'DLC_Program', 'FEM_Program', 'HM_Circular_Program',
       'Internal_Decor', 'Interior_Decor_Signage_Available_Funds',
       'LMS_Enrolled', 'LMS_Available_Matching_Funds',
       'LMS_Used_Funds_Most_Recent_3_Months',
       'LMS_Used_Funds_Previous_3_Months', 'Open_Smart',
       'MRA_Program_Type_-_Active_AH', 'Msa_Dma', 'OTC_Front-end_Size',
       'PIP_Program', 'Pog_Code_Name', 'KYN_Training', 'PQS_Enrolled',
       'PQS_Program', 'Last_EQuIPP_Login', 

In [118]:
# unnamed_cols = df_clean.select(lambda col: col.startswith('Unnamed'), axis=1)
# unnamed_cols.columns

In [262]:
# df_raw = df_raw.drop(['Unnamed: 86', 'Unnamed: 87', 'Unnamed: 88', 'Unnamed: 89'], axis=1)

In [358]:
df_raw.Internal_Decor.unique()

array(['X', nan], dtype=object)

In [360]:
df_clean = df_raw

In [ ]:
def fill_in_missing_dates(df, date_col_name = 'date',date_order = 'asc', fill_value = 0, days_back = 30):

    df.set_index(date_col_name,drop=True,inplace=True)
    df.index = pd.DatetimeIndex(df.index)
    d = datetime.now().date()
    d2 = d - timedelta(days = days_back)
    idx = pd.date_range(d2, d, freq = "D")
    df = df.reindex(idx,fill_value=fill_value)
    df[date_col_name] = pd.DatetimeIndex(df.index)

    return df

In [361]:
# if you want to operate on multiple columns, put them in a list like so:
dollar_cols = ['Interior_Decor_Signage_Available_Funds', 'LMS_Available_Matching_Funds', 'Tot_Sls_Amt', 'Brand_Rx_Sls_Amt', 
               'Gnrc_Rx_Sls_Amt','Os_Gnrc_Rx_Sls_Amt','Otc_Sls_Amt', 'LMS_Enrolled', 'LMS_Available_Matching_Funds', 
               'Vaccine_Items_Sls_Amt']
X_cols = ['Internal_Decor', 'Open_Smart', 'PIP_Program', 'PQS_Program', 'SCS_Program', 'Specialty_Solutions', 'Vaccine_Items',
          'Vaccine_Starter', 'YPO']
y_n_cols = ['340B_Active', 'LMS_Enrolled', 'LMS_Used_Funds_Most_Recent_3_Months', 'LMS_Used_Funds_Previous_3_Months']
yes_cols = ['Auto_Ship_Ind', 'Spar_Coverage_Ind']
empt_num_cols = ['Profitability', 'Annual_Dispense_Fee_Total', 'Script_Count', 'Macro_PID', 'Capture_Rate', 'Specialty_Spend',
                'Specialty_Product_Purchases', 'Generic', 'Brand', 'Generic_Ratio', 'Vaccine_Items_Count']
date_cols = ['Hm_Start_Dt','KYN_Training', 'Last_EQuIPP_Login', 'PQS_Enrolled_Date'] 
df_clean[X_cols] = df_clean[X_cols].applymap(lambda x: 1 if x == 'X' else 0)
df_clean[y_n_cols] = df_clean[y_n_cols].applymap(lambda x: 1 if x == 'Y' else 0)
df_clean[yes_cols] = df_clean[yes_cols].applymap(lambda x: 1 if x == 'Yes' else 0)
df_clean['3rd_Party_Vendor'] = df_clean['3rd_Party_Vendor'].apply(lambda x: 'UNK' if pd.isnull(x) else x).astype(str)
df_clean['Hospital_Associated'] = df_clean['Hospital_Associated'].apply(lambda x: 'UNK' if pd.isnull(x) else x).astype(str)
df_clean['IS_HID'] = df_clean['HID'].apply(lambda x: 0 if pd.isnull(x) else 1)
df_clean[empt_num_cols] = df_clean[empt_num_cols].applymap(lambda x: 0 if pd.isnull(x) else x)
df_clean['DLC_Program'] = df_clean['DLC_Program'].apply(lambda x: 'DLC_Program' if x == 'X' else ('Unknown' if pd.isnull(x) else x))
df_clean['FEM_Program'] = df_clean['FEM_Program'].apply(lambda x: 'FEM_Program' if x == 'X' else ('Unknown' if pd.isnull(x) else x))
df_clean['HM_Circular_Program'] = df_clean['HM_Circular_Program'].apply(lambda x: 'HM_Circular_Program' if x == 'X' else ('Unknown' if pd.isnull(x) else x))
df_clean[dollar_cols]=df_clean[dollar_cols].fillna(0)
df_clean[date_cols]=df_clean[date_cols].fillna(pd.Timestamp('19740101'))
df_clean['Target']=df_clean['Target'].fillna(0)


In [210]:
# df_clean['Capture_Rate'].unique
# df_clean['Capture_Rate'].value_counts()
# df_clean['DLC Program'].isnull().values.any()
# df_raw['3rd Party Vendor'] = df_raw['3rd Party Vendor'].astype(str)
# tmp = df_raw['3rd Party Vendor'].fillna(''); isEmpty = tmp==''
# tmp

In [265]:
# df_raw['3rd Party Vendor'].dtype
df_clean.head(5)

,Nabp_Ncpdp,Npi_Num,Dea_Num,Target,Store_Name,Address-Delivery,City-Delivery,ST,Zip,Owner_Mgr_Full_Name,...,Profitability,Annual_Dispense_Fee_Total,Macro_PID,Script_Count,Capture_Rate,Specialty_Spend,Specialty_Product_Purchases,Generic,Brand,Generic_Ratio
Account,,,,,,,,,,,,,,,,,,,,,
10491,3800047,1205901485,AM1635474,10491.0,POSTAL PHARMACY,47809 HIGHWAY 58,OAKRIDGE,OR,97463,Laurie Patty,...,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0
17475,3802421,1912059569,AH1621336,17475.0,HOWARDS DRUGS,101 NORTH F STREET,LAKEVIEW,OR,97630,Jeff Howard,...,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0
19901,3804425,1114456563,AK1605887,19901.0,HARVARD AVENUE DRUGS,1175 HARVARD AVE,ROSEBURG,OR,97471,Philip Kaser,...,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0
309745,3804324,1437294808,FS5689964,309745.0,REEDSPORT PHARMACY,1409 HWY 101,REEDSPORT,OR,97467,JAMAICA STANDIFORD,...,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0
652229,3844366,1275808743,FJ3131290,652229.0,PHILOMATH FAMILY PHARMACY,1640 MAIN STREET,PHILOMATH,OR,97370,SETH PHAN,...,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0


In [15]:
# df_clean = df_raw[['Account','Store Name','AddressDelivery','CityDelivery','ST','Zip','Region','DC Name','PSAO Expanded',
#                    'Buying Group','AH Program','Auto Ship Ind','DLC Program','FEM Program','HM Circular Program',
#                    'Internal Decor','Interior Decor / Signage Available Funds','LMS Enrolled','LMS Available Matching Funds',
#                    'LMS Used Funds Most Recent 3 Months','LMS Used Funds Previous 3 Months','Open Smart',
#                    'MRA Program Type  Active AH','OTC Frontend Size','PIP Program','Pog Code Name', 'PQS Enrolled',
#                    'Last EQuIPP Login','SCS Program','Specialty Solutions','Vaccine Items','Vaccine Starter',
#                    'Vaccine Items Count','Vaccine Items Sls Amt','YPO','Tot Sls Amt','Brand Rx Sls Amt','Gnrc Rx Sls Amt',
#                    'Os Gnrc Rx Sls Amt','Otc Sls Amt','3rd Party Vendor','Hospital Associated','340B Active']]
# df_clean.head()

In [318]:
# df_clean['LMS_Enrolled'].value_counts()
df_clean['LMS_Enrolled'].unique()

array([1, 0], dtype=int64)

In [64]:
# df_clean['Interior Decor / Signage Available Funds'].value_counts

In [319]:
df_clean.columns

Index(['Nabp_Ncpdp', 'Npi_Num', 'Dea_Num', 'Target', 'Store_Name',
       'Address-Delivery', 'City-Delivery', 'ST', 'Zip', 'Owner_Mgr_Full_Name',
       'Owner_Email', 'Phone_Number', 'Hm_Start_Dt', 'Channel_Type_Cd',
       'Cust_Type_Desc', 'Bus_Type_Desc', 'Region', 'DC_Name', 'PSAO_Expanded',
       'Buying_Group', 'Chain_Name', 'Sub_Group_Name', 'Region_Name',
       'Region_Director', 'Pharm_D', 'Vpgm_Name', 'Vps_Name', 'Dsm_Name',
       'RSM_Name', 'Salesperson', 'Salesperson_Group', 'AH_Program',
       'Auto_Ship_Ind', 'DLC_Program', 'FEM_Program', 'HM_Circular_Program',
       'Internal_Decor', 'Interior_Decor_Signage_Available_Funds',
       'LMS_Enrolled', 'LMS_Available_Matching_Funds',
       'LMS_Used_Funds_Most_Recent_3_Months',
       'LMS_Used_Funds_Previous_3_Months', 'Open_Smart',
       'MRA_Program_Type_-_Active_AH', 'Msa_Dma', 'OTC_Front-end_Size',
       'PIP_Program', 'Pog_Code_Name', 'KYN_Training', 'PQS_Enrolled',
       'PQS_Program', 'Last_EQuIPP_Login', 

In [362]:
# to drop from df_clean
drop_cols = ['Nabp_Ncpdp','Npi_Num','Dea_Num','Target','Store_Name','Address-Delivery','City-Delivery','Cust_Type_Desc', 
             'Owner_Mgr_Full_Name', 'Owner_Email', 'Phone_Number','Sub_Group_Name','Region_Director','Pharm_D','Vpgm_Name',
             'Vps_Name','Dsm_Name', 'RSM_Name','Salesperson_Group', 'Zip_of_HID','HID','Zip_Match_Level','340B_ID']
df_clean = df_clean.drop(drop_cols, axis=1)


In [140]:
# df_clean = df_clean[['ST','Zip','Hm_Start_Dt','Channel_Type_Cd','Cust_Type_Desc','Bus_Type_Desc','Region','DC_Name',
#                      'PSAO_Expanded','Buying_Group','Chain_Name','Region_Name','Salesperson','AH_Program','Auto_Ship_Ind',
#                      'DLC_Program','FEM_Program','HM_Circular_Program','Internal_Decor','Interior_Decor_Signage_Available_Funds',
#                      'LMS_Enrolled','LMS_Available_Matching_Funds','LMS_Used_Funds_Most_Recent_3_Months',
#                      'LMS_Used_Funds_Previous_3_Months','Open_Smart','MRA_Program_Type_-_Active_AH','Msa_Dma',
#                      'OTC_Frontend_Size','PIP_Program','Pog_Code_Name','KYN_Training','PQS_Enrolled','PQS_Program',
#                      'Last_EQuIPP_Login','PQS_Enrolled_Date','SCS_Program','Signage_Program','Spar_Coverage_Ind',
#                      'Specialty_Solutions','Vaccine_Items','Vaccine_Starter','Vaccine_Items_Count','Vaccine_Items_Sls_Amt','YPO',
#                      'Tot_Sls_Amt','Brand_Rx_Sls_Amt','Gnrc_Rx_Sls_Amt','Os_Gnrc_Rx_Sls_Amt','Otc_Sls_Amt','HID',
#                      '3rd_Party_Vendor','Hospital_Associated','340B_Active','Profitability','Annual_Dispense_Fee_Total',
#                      'Macro_PID','Script_Count','Capture_Rate','Specialty_Spend','Specialty_Product_Purchases','Generic',
#                      'Brand','Generic_Ratio']]

In [277]:
df_clean.columns

Index(['ST', 'Zip', 'Hm_Start_Dt', 'Channel_Type_Cd', 'Bus_Type_Desc',
       'Region', 'DC_Name', 'PSAO_Expanded', 'Buying_Group', 'Chain_Name',
       'Region_Name', 'Salesperson', 'AH_Program', 'Auto_Ship_Ind',
       'DLC_Program', 'FEM_Program', 'HM_Circular_Program', 'Internal_Decor',
       'Interior_Decor_Signage_Available_Funds', 'LMS_Enrolled',
       'LMS_Available_Matching_Funds', 'LMS_Used_Funds_Most_Recent_3_Months',
       'LMS_Used_Funds_Previous_3_Months', 'Open_Smart',
       'MRA_Program_Type_-_Active_AH', 'Msa_Dma', 'OTC_Front-end_Size',
       'PIP_Program', 'Pog_Code_Name', 'KYN_Training', 'PQS_Enrolled',
       'PQS_Program', 'Last_EQuIPP_Login', 'PQS_Enrolled_Date', 'SCS_Program',
       'Signage_Program', 'Spar_Coverage_Ind', 'Specialty_Solutions',
       'Vaccine_Items', 'Vaccine_Starter', 'Vaccine_Items_Count',
       'Vaccine_Items_Sls_Amt', 'YPO', 'Tot_Sls_Amt', 'Brand_Rx_Sls_Amt',
       'Gnrc_Rx_Sls_Amt', 'Os_Gnrc_Rx_Sls_Amt', 'Otc_Sls_Amt', 'IS_HID',
   

In [363]:
df_clean[date_cols]=df_clean[date_cols].applymap(dt.datetime.toordinal)
# these cat_cols need to have categorized data so we can include them in the analysis
cat_cols = ['DLC_Program', 'FEM_Program', 'HM_Circular_Program', 'PQS_Enrolled', 'ST', 'Pog_Code_Name',
            'DC_Name', 'Bus_Type_Desc', 'Region', 'DC_Name', 'PSAO_Expanded', 'Buying_Group', 'Chain_Name', 'Region_Name', 
            'Salesperson', 'AH_Program', 'OTC_Front-end_Size', 'MRA_Program_Type_-_Active_AH', 'Msa_Dma', 'Signage_Program', 
            '3rd_Party_Vendor', 'Hospital_Associated']
le = LabelEncoder()
# df_clean['ST_encoded'] = le.fit_transform(df_clean['ST'])
# df_clean['DLC_Program_encoded'] = le.fit_transform(df_clean['DLC_Program'])
for lbl in cat_cols:
    new_la = lbl + '_encoded'
    df_clean[new_la] = le.fit_transform(df_clean[lbl].astype(str))
# drop_enc_cols = (['DLC_Program', 'FEM_Program', 'HM_Circular_Program', 'Internal_Decor', 'PQS_Enrolled', 'ST', 'Pog_Code_Name',
#                     'DC_Name', 'Bus_Type_Desc', 'Region', 'PSAO_Expanded', ])
df_clean = df_clean.drop(cat_cols, axis=1)
df_clean.head(5)

,Zip,Hm_Start_Dt,Channel_Type_Cd,Auto_Ship_Ind,Internal_Decor,Interior_Decor_Signage_Available_Funds,LMS_Enrolled,LMS_Available_Matching_Funds,LMS_Used_Funds_Most_Recent_3_Months,LMS_Used_Funds_Previous_3_Months,...,Chain_Name_encoded,Region_Name_encoded,Salesperson_encoded,AH_Program_encoded,OTC_Front-end_Size_encoded,MRA_Program_Type_-_Active_AH_encoded,Msa_Dma_encoded,Signage_Program_encoded,3rd_Party_Vendor_encoded,Hospital_Associated_encoded
Account,,,,,,,,,,,,,,,,,,,,,
10491,97463,732524,30,1,1,1724.0,1,822.0,1,1,...,39,23,1,4,1,7,183,0,18,15
17475,97630,732687,30,1,1,0.0,1,0.0,1,1,...,39,23,1,0,3,3,190,0,18,15
19901,97471,732616,30,0,1,0.0,1,0.0,1,1,...,39,23,1,0,4,2,183,0,18,15
309745,97467,735998,30,0,0,2500.0,1,0.0,0,0,...,39,23,1,0,1,8,183,3,18,15
652229,97370,734702,30,0,1,0.0,1,340.0,1,0,...,38,23,1,0,0,8,183,0,18,15


In [23]:
# label_encoder=LabelEncoder()
# X['DLC Program'] = labelencoder_X.fit_transform(X['DLC Program'])
# df["ModelNumeric"] = df["Model"].astype("category", ordered = True).cat.codes # Thank you B.Turnwald

In [227]:
# df_clean['Target'].unique()
# df_clean.isnull().head(6)
# df_clean.isnull().values.any()

null_columns=df_clean.columns[df_clean.isnull().any()]
df_clean[null_columns].isnull().sum()


Series([], dtype: float64)

In [21]:
# print(df_raw.loc[df_raw['340B Active'] == 1])
# df_is = df_clean.loc[df_clean['340B Active'] == 1]
# df_is.head()

In [284]:
df_clean.head()

,Zip,Hm_Start_Dt,Channel_Type_Cd,Auto_Ship_Ind,Interior_Decor_Signage_Available_Funds,LMS_Enrolled,LMS_Available_Matching_Funds,LMS_Used_Funds_Most_Recent_3_Months,LMS_Used_Funds_Previous_3_Months,Open_Smart,...,Chain_Name_encoded,Region_Name_encoded,Salesperson_encoded,AH_Program_encoded,OTC_Front-end_Size_encoded,MRA_Program_Type_-_Active_AH_encoded,Msa_Dma_encoded,Signage_Program_encoded,3rd_Party_Vendor_encoded,Hospital_Associated_encoded
Account,,,,,,,,,,,,,,,,,,,,,
10491,97463,2006-08-01,30,0,1724.0,0,822.0,0,0,0,...,39,23,1,4,1,7,183,0,18,15
17475,97630,2007-01-11,30,0,0.0,0,0.0,0,0,0,...,39,23,1,0,3,3,190,0,18,15
19901,97471,2006-11-01,30,0,0.0,0,0.0,0,0,0,...,39,23,1,0,4,2,183,0,18,15
309745,97467,2016-02-04,30,0,2500.0,0,0.0,0,0,0,...,39,23,1,0,1,8,183,3,18,15
652229,97370,2012-07-18,30,0,0.0,0,340.0,0,0,0,...,38,23,1,0,0,8,183,0,18,15


In [285]:
df_clean.columns

Index(['Zip', 'Hm_Start_Dt', 'Channel_Type_Cd', 'Auto_Ship_Ind',
       'Interior_Decor_Signage_Available_Funds', 'LMS_Enrolled',
       'LMS_Available_Matching_Funds', 'LMS_Used_Funds_Most_Recent_3_Months',
       'LMS_Used_Funds_Previous_3_Months', 'Open_Smart', 'PIP_Program',
       'KYN_Training', 'PQS_Program', 'Last_EQuIPP_Login', 'PQS_Enrolled_Date',
       'SCS_Program', 'Spar_Coverage_Ind', 'Specialty_Solutions',
       'Vaccine_Items', 'Vaccine_Starter', 'Vaccine_Items_Count',
       'Vaccine_Items_Sls_Amt', 'YPO', 'Tot_Sls_Amt', 'Brand_Rx_Sls_Amt',
       'Gnrc_Rx_Sls_Amt', 'Os_Gnrc_Rx_Sls_Amt', 'Otc_Sls_Amt', 'IS_HID',
       '340B_Active', 'Profitability', 'Annual_Dispense_Fee_Total',
       'Macro_PID', 'Script_Count', 'Capture_Rate', 'Specialty_Spend',
       'Specialty_Product_Purchases', 'Generic', 'Brand', 'Generic_Ratio',
       'DLC_Program_encoded', 'FEM_Program_encoded',
       'HM_Circular_Program_encoded', 'Internal_Decor_encoded',
       'PQS_Enrolled_encoded',

In [299]:
# df_clean.loc[:, df_clean.dtypes == object]
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4846 entries, 10491 to 994517
Data columns (total 62 columns):
Zip                                       4846 non-null int64
Hm_Start_Dt                               4846 non-null int64
Channel_Type_Cd                           4846 non-null int64
Auto_Ship_Ind                             4846 non-null int64
Interior_Decor_Signage_Available_Funds    4846 non-null float64
LMS_Enrolled                              4846 non-null int64
LMS_Available_Matching_Funds              4846 non-null float64
LMS_Used_Funds_Most_Recent_3_Months       4846 non-null int64
LMS_Used_Funds_Previous_3_Months          4846 non-null int64
Open_Smart                                4846 non-null int64
PIP_Program                               4846 non-null int64
KYN_Training                              4846 non-null int64
PQS_Program                               4846 non-null int64
Last_EQuIPP_Login                         4846 non-null int64
PQS_Enrolled_Da

In [364]:
# df_clean['IS_HID'].unique()


In [366]:
X = df_clean[[
       'Zip', 'Hm_Start_Dt', 'Channel_Type_Cd', 'Auto_Ship_Ind',
       'Interior_Decor_Signage_Available_Funds', 'LMS_Enrolled',
       'LMS_Available_Matching_Funds', 'LMS_Used_Funds_Most_Recent_3_Months',
       'LMS_Used_Funds_Previous_3_Months', 'Open_Smart', 'PIP_Program',
       'KYN_Training', 'PQS_Program', 'Last_EQuIPP_Login', 'PQS_Enrolled_Date',
       'SCS_Program', 'Spar_Coverage_Ind', 'Specialty_Solutions',
       'Vaccine_Items', 'Vaccine_Starter', 'Vaccine_Items_Count',
       'Vaccine_Items_Sls_Amt', 'YPO', 'Tot_Sls_Amt', 'Brand_Rx_Sls_Amt',
       'Gnrc_Rx_Sls_Amt', 'Os_Gnrc_Rx_Sls_Amt', 'Otc_Sls_Amt', 
       '340B_Active', 'Profitability', 'Annual_Dispense_Fee_Total',
       'Macro_PID', 'Script_Count', 'Capture_Rate', 'Specialty_Spend',
       'Specialty_Product_Purchases', 'Generic', 'Brand', 'Generic_Ratio',
       'DLC_Program_encoded', 'FEM_Program_encoded',
       'HM_Circular_Program_encoded', 'Internal_Decor',
       'PQS_Enrolled_encoded', 'ST_encoded', 'Pog_Code_Name_encoded',
       'DC_Name_encoded', 'Bus_Type_Desc_encoded', 'Region_encoded',
       'PSAO_Expanded_encoded', 'Buying_Group_encoded', 'Chain_Name_encoded',
       'Region_Name_encoded', 'Salesperson_encoded', 'AH_Program_encoded',
       'OTC_Front-end_Size_encoded', 'MRA_Program_Type_-_Active_AH_encoded',
       'Msa_Dma_encoded', 'Signage_Program_encoded',
       '3rd_Party_Vendor_encoded', 'Hospital_Associated_encoded']]

y = df_clean['IS_HID'] 
# X = sm.add_constant(X)
# lr = sm.OLS(y, X)
# lr = lr.fit()
# lr.summary()  
logit = sm.Logit(y, X)
logit.fit()
logit.summary()
# model = LogisticRegression(fit_intercept=False)
# mdl = model.fit(X,y)
# model.coef_


         Current function value: inf
         Iterations: 35


C:\Users\elkzze1\anaconda\anaconda3\envs\py3\lib\site-packages\statsmodels\discrete\discrete_model.py:1214: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-X))
C:\Users\elkzze1\anaconda\anaconda3\envs\py3\lib\site-packages\statsmodels\discrete\discrete_model.py:1264: RuntimeWarning: divide by zero encountered in log
  return np.sum(np.log(self.cdf(q*np.dot(X,params))))


LinAlgError: Singular matrix

In [330]:
X.head(10)

,Zip,Hm_Start_Dt,Channel_Type_Cd,Auto_Ship_Ind,Interior_Decor_Signage_Available_Funds,LMS_Enrolled,LMS_Available_Matching_Funds,LMS_Used_Funds_Most_Recent_3_Months,LMS_Used_Funds_Previous_3_Months,Open_Smart,...,Chain_Name_encoded,Region_Name_encoded,Salesperson_encoded,AH_Program_encoded,OTC_Front-end_Size_encoded,MRA_Program_Type_-_Active_AH_encoded,Msa_Dma_encoded,Signage_Program_encoded,3rd_Party_Vendor_encoded,Hospital_Associated_encoded
Account,,,,,,,,,,,,,,,,,,,,,
10491,97463,732524,30,1,1724.0,1,822.0,1,1,0,...,39,23,1,4,1,7,183,0,18,15
17475,97630,732687,30,1,0.0,1,0.0,1,1,0,...,39,23,1,0,3,3,190,0,18,15
19901,97471,732616,30,0,0.0,1,0.0,1,1,0,...,39,23,1,0,4,2,183,0,18,15
309745,97467,735998,30,0,2500.0,1,0.0,0,0,0,...,39,23,1,0,1,8,183,3,18,15
652229,97370,734702,30,0,0.0,1,340.0,1,0,0,...,38,23,1,0,0,8,183,0,18,15
121439,49242,735369,30,1,2500.0,1,2415.0,1,1,0,...,22,23,2,0,4,2,51,1,18,10
9948,43606,734825,30,1,0.0,1,2233.0,1,1,0,...,36,40,2,0,1,3,48,1,18,15
13973,48101,735510,30,0,2500.0,1,1200.0,0,0,0,...,36,40,2,0,6,8,6,2,18,15
18052,48033,735206,30,0,0.0,1,1200.0,0,0,0,...,36,40,2,0,0,8,6,0,18,15


In [67]:
Xreduced =X.drop('DC_Name_encoded', axis=1)
y = df_clean['Is_HID'] 
Xreduced = sm.add_constant(Xreduced)
# lr = sm.OLS(y, X)
# lr = lr.fit()
# lr.summary()  
logit = sm.Logit(y, Xreduced)
logit = logit.fit()
logit.summary()

Optimization terminated successfully.
         Current function value: 0.148696
         Iterations 8


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                 Is_HID   No. Observations:                 4846
Model:                          Logit   Df Residuals:                     4841
Method:                           MLE   Df Model:                            4
Date:                Fri, 09 Feb 2018   Pseudo R-squ.:                 0.05131
Time:                        09:38:54   Log-Likelihood:                -720.58
converged:                       True   LL-Null:                       -759.55
                                        LLR p-value:                 4.736e-16
==========================================================================================
                             coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------
const                     -2.3859      0.187    -12.781      0.000      -2.752      -2.020
ST_encoded                -0.0116      0.006     -1.890      0.059      -0.024       0.000
Cust_Type_Desc_encoded     0.3800      0.165      2.302      0.021       0.056       0.704
Bus_Type_Desc_encoded      0.0646      0.034      1.918      0.055      -0.001       0.131
Region_encoded            -0.6015      0.079     -7.601      0.000      -0.757      -0.446
==========================================================================================
"""

In [68]:
for i, col in enumerate(Xreduced.columns):            # check VIF
    print('VIF {}: {}'.format(col,variance_inflation_factor(Xreduced.values,i)))

VIF const: 6.667849880210776
VIF ST_encoded: 1.0640032290937613
VIF Cust_Type_Desc_encoded: 1.139318688103975
VIF Bus_Type_Desc_encoded: 1.135484046557424
VIF Region_encoded: 1.0682091411694272


In [350]:
for i, col in enumerate(X.columns):            # check VIF
    print('VIF {}: {}'.format(col,variance_inflation_factor(X.values,i)))

VIF Zip: 17.614194402718223
VIF Hm_Start_Dt: 159843.7231119183
VIF Channel_Type_Cd: 860.8139426158743
VIF Auto_Ship_Ind: 2.1434285311622454
VIF Interior_Decor_Signage_Available_Funds: 4.27954965074896
VIF LMS_Enrolled: 7.99796606074536
VIF LMS_Available_Matching_Funds: 1.6578864787827778
VIF LMS_Used_Funds_Most_Recent_3_Months: 3.4351349991336066
VIF LMS_Used_Funds_Previous_3_Months: 3.403048190387261
VIF Open_Smart: 1.3499271928070378
VIF PIP_Program: 2.195687128543567
VIF KYN_Training: 15533.745070371348
VIF PQS_Program: 10.848731541882753
VIF Last_EQuIPP_Login: 20620.376911369483
VIF PQS_Enrolled_Date: 153079.1138312649
VIF SCS_Program: 4.111491643291372
VIF Spar_Coverage_Ind: 4.694951814905574
VIF Specialty_Solutions: 1.1303681659964746
VIF Vaccine_Items: 2.7337585643966458
VIF Vaccine_Starter: 1.0709889686787453
VIF Vaccine_Items_Count: 3.0663724839477955
VIF Vaccine_Items_Sls_Amt: 2.2748494864080424
VIF YPO: 1.6247673281726147
VIF Tot_Sls_Amt: 276574423641.75366
VIF Brand_Rx_Sls_

C:\Users\elkzze1\anaconda\anaconda3\envs\py3\lib\site-packages\statsmodels\regression\linear_model.py:1388: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - self.ssr/self.uncentered_tss


VIF Internal_Decor_encoded: nan
VIF PQS_Enrolled_encoded: 9.111240604151858
VIF ST_encoded: 3.943976860806179
VIF Pog_Code_Name_encoded: 5.047773664732984
VIF DC_Name_encoded: 5.037225275265013
VIF Bus_Type_Desc_encoded: 1.124090865534397
VIF Region_encoded: 5.052893676641578
VIF PSAO_Expanded_encoded: 3.3454482241544703
VIF Buying_Group_encoded: 15.074455860308287
VIF Chain_Name_encoded: 18.749098000779753
VIF Region_Name_encoded: 17.05000040551862
VIF Salesperson_encoded: 4.181981385432791
VIF AH_Program_encoded: 3.742719056292036
VIF OTC_Front-end_Size_encoded: 5.8685267748862024
VIF MRA_Program_Type_-_Active_AH_encoded: 9.503448385239933
VIF Msa_Dma_encoded: 10.836198833037002
VIF Signage_Program_encoded: 3.505976850270867
VIF 3rd_Party_Vendor_encoded: 41.69433367099083
VIF Hospital_Associated_encoded: 15.712842930847566


In [77]:
# X1 = X.drop('INSERT COLUMN NAME HERE', axis=1)
# for i, col in enumerate(X1.columns):            # check VIF
#     print('VIF {}: {}'.format(col,variance_inflation_factor(X1.values,i)))

In [ ]:
for i, col in enumerate(X1.columns):
    print('VIF {}: {}'.format(col,variance_inflation_factor(X1.values,i))) 
X2 = X.drop('INSERT COLUMN NAME HERE', axis=1) 

In [ ]:
for i, col in enumerate(X2.columns):
    print('VIF {}: {}'.format(col,variance_inflation_factor(X2.values,i))) 

In [70]:
# need to do test_train_split

X_train, X_test, y_train, y_test = train_test_split(Xreduced, y, test_size=0.33, random_state=42)

In [71]:
# random forest

rf = RandomForestClassifier()
rf.fit(X_train, y_train)
rf.feature_importances_
rf.predict(X_test) 


array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [74]:
pred_rf = rf.predict(X_test)
pred_rf

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [72]:
# decision tree

dt = DecisionTreeClassifier()
dt.fit(X_train, y_train)
dt.feature_importances_
dt.predict(X_test) 

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [76]:
pred_dt = dt.predict(X_test)
pred_dt

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [ ]:
# some way to plot pred_rf against y_test
# and plot pred_dt against y_test

In [ ]:
# and can do r2_score of the rf and then dt
# i don't know if logit (logistic regression) has an r2?
# trying to beat an r2 of .69

In [79]:
scores = cross_val_score(rf, X_train, y_train, scoring='r2', cv=5)
scores

array([-0.08160277, -0.03833866, -0.12493333, -0.0384    , -0.39742609])

In [ ]:
# feature importances

In [181]:

# X3 = df_clean.drop()

y = df_clean['IS_HID'] 
X = sm.add_constant(X3)
# lr = sm.OLS(y, X)
# lr = lr.fit()
# lr.summary()  
logit = sm.Logit(y, X3)
logit = logit.fit()
logit.summary()

ValueError: Pandas data cast to numpy dtype of object. Check input data with np.asarray(data).

In [179]:
df_clean.columns

Index(['ST', 'Zip', 'Hm_Start_Dt', 'Channel_Type_Cd', 'Bus_Type_Desc',
       'Region', 'DC_Name', 'PSAO_Expanded', 'Buying_Group', 'Chain_Name',
       'Region_Name', 'Salesperson', 'AH_Program', 'Auto_Ship_Ind',
       'DLC_Program', 'FEM_Program', 'HM_Circular_Program', 'Internal_Decor',
       'Interior_Decor_Signage_Available_Funds', 'LMS_Enrolled',
       'LMS_Available_Matching_Funds', 'LMS_Used_Funds_Most_Recent_3_Months',
       'LMS_Used_Funds_Previous_3_Months', 'Open_Smart',
       'MRA_Program_Type_-_Active_AH', 'Msa_Dma', 'OTC_Front-end_Size',
       'PIP_Program', 'Pog_Code_Name', 'KYN_Training', 'PQS_Enrolled',
       'PQS_Program', 'Last_EQuIPP_Login', 'PQS_Enrolled_Date', 'SCS_Program',
       'Signage_Program', 'Spar_Coverage_Ind', 'Specialty_Solutions',
       'Vaccine_Items', 'Vaccine_Starter', 'Vaccine_Items_Count',
       'Vaccine_Items_Sls_Amt', 'YPO', 'Tot_Sls_Amt', 'Brand_Rx_Sls_Amt',
       'Gnrc_Rx_Sls_Amt', 'Os_Gnrc_Rx_Sls_Amt', 'Otc_Sls_Amt', 'HID', 'IS_HI